## Car Retail Price Prediction

Given *data about various cars*, let's try to predict the **suggested retail price** of a given car.

We will use a variety of regression models to make our predictions.

Data source: https://www.kaggle.com/datasets/CooperUnion/cardataset

### Getting Started

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [2]:
data = pd.read_csv('archive/data.csv')
data

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

### Preprocessing

In [4]:
df = data.copy()

In [5]:
df

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [6]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'Make': 48,
 'Model': 915,
 'Engine Fuel Type': 11,
 'Transmission Type': 5,
 'Driven_Wheels': 4,
 'Market Category': 72,
 'Vehicle Size': 3,
 'Vehicle Style': 16}

In [7]:
{column: df[column].unique() for column in df.select_dtypes('object').columns}

{'Make': array(['BMW', 'Audi', 'FIAT', 'Mercedes-Benz', 'Chrysler', 'Nissan',
        'Volvo', 'Mazda', 'Mitsubishi', 'Ferrari', 'Alfa Romeo', 'Toyota',
        'McLaren', 'Maybach', 'Pontiac', 'Porsche', 'Saab', 'GMC',
        'Hyundai', 'Plymouth', 'Honda', 'Oldsmobile', 'Suzuki', 'Ford',
        'Cadillac', 'Kia', 'Bentley', 'Chevrolet', 'Dodge', 'Lamborghini',
        'Lincoln', 'Subaru', 'Volkswagen', 'Spyker', 'Buick', 'Acura',
        'Rolls-Royce', 'Maserati', 'Lexus', 'Aston Martin', 'Land Rover',
        'Lotus', 'Infiniti', 'Scion', 'Genesis', 'HUMMER', 'Tesla',
        'Bugatti'], dtype=object),
 'Model': array(['1 Series M', '1 Series', '100', '124 Spider', '190-Class',
        '2 Series', '200', '200SX', '240SX', '240', '2',
        '3 Series Gran Turismo', '3 Series', '300-Class', '3000GT', '300',
        '300M', '300ZX', '323', '350-Class', '350Z', '360', '370Z', '3',
        '4 Series Gran Coupe', '4 Series', '400-Class', '420-Class',
        '456M', '458 Italia', '4C'

In [8]:
# Drop high cardinality columns
df = df.drop('Model', axis=1)

In [9]:
# Fill multi hot column missing values
df['Market Category'] = df['Market Category'].fillna('Missing')

In [10]:
def multihot_encode(df, column):
    df = df.copy()

    df[column] = df[column].apply(lambda x: x.split(','))
    
    all_categories = np.unique(df[column].sum())

    for category in all_categories:
        df[column + '_' + category] = df.apply(lambda x: 1 if category in x[column] else 0, axis=1)

    df = df.drop(column, axis=1)
    
    return df

In [11]:
# Multi-hot encoding
df = multihot_encode(df, 'Market Category')

In [12]:
df

,Make,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Market Category_Crossover,Market Category_Diesel,Market Category_Exotic,Market Category_Factory Tuner,Market Category_Flex Fuel,Market Category_Hatchback,Market Category_High-Performance,Market Category_Hybrid,Market Category_Luxury,Market Category_Missing,Market Category_Performance
0,BMW,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,26,19,3916,46135,0,0,0,1,0,0,1,0,1,0,0
1,BMW,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,19,3916,40650,0,0,0,0,0,0,0,0,1,0,1
2,BMW,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,20,3916,36350,0,0,0,0,0,0,1,0,1,0,0
3,BMW,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,18,3916,29450,0,0,0,0,0,0,0,0,1,0,1
4,BMW,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,18,3916,34500,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,46120,1,0,0,0,0,1,0,0,1,0,0
11910,Acura,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,56670,1,0,0,0,0,1,0,0,1,0,0
11911,Acura,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,50620,1,0,0,0,0,1,0,0,1,0,0
11912,Acura,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,50920,1,0,0,0,0,1,0,0,1,0,0


In [13]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [14]:
# One-hot encoding
for column in df.select_dtypes('object').columns:
    df = onehot_encode(df, column=column)

In [15]:
df

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,MSRP,Market Category_Crossover,Market Category_Diesel,Market Category_Exotic,Market Category_Factory Tuner,Market Category_Flex Fuel,Market Category_Hatchback,Market Category_High-Performance,Market Category_Hybrid,Market Category_Luxury,Market Category_Missing,Market Category_Performance,Make_Acura,Make_Alfa Romeo,Make_Aston Martin,Make_Audi,Make_BMW,Make_Bentley,Make_Bugatti,Make_Buick,Make_Cadillac,Make_Chevrolet,Make_Chrysler,Make_Dodge,Make_FIAT,Make_Ferrari,Make_Ford,Make_GMC,Make_Genesis,Make_HUMMER,Make_Honda,Make_Hyundai,Make_Infiniti,Make_Kia,Make_Lamborghini,Make_Land Rover,Make_Lexus,Make_Lincoln,Make_Lotus,Make_Maserati,Make_Maybach,Make_Mazda,Make_McLaren,Make_Mercedes-Benz,Make_Mitsubishi,Make_Nissan,Make_Oldsmobile,Make_Plymouth,Make_Pontiac,Make_Porsche,Make_Rolls-Royce,Make_Saab,Make_Scion,Make_Spyker,Make_Subaru,Make_Suzuki,Make_Tesla,Make_Toyota,Make_Volkswagen,Make_Volvo,Engine Fuel Type_diesel,Engine Fuel Type_electric,Engine Fuel Type_flex-fuel (premium unleaded recommended/E85),Engine Fuel Type_flex-fuel (premium unleaded required/E85),Engine Fuel Type_flex-fuel (unleaded/E85),Engine Fuel Type_flex-fuel (unleaded/natural gas),Engine Fuel Type_natural gas,Engine Fuel Type_premium unleaded (recommended),Engine Fuel Type_premium unleaded (required),Engine Fuel Type_regular unleaded,Transmission Type_AUTOMATED_MANUAL,Transmission Type_AUTOMATIC,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize,Vehicle Style_2dr Hatchback,Vehicle Style_2dr SUV,Vehicle Style_4dr Hatchback,Vehicle Style_4dr SUV,Vehicle Style_Cargo Minivan,Vehicle Style_Cargo Van,Vehicle Style_Convertible,Vehicle Style_Convertible SUV,Vehicle Style_Coupe,Vehicle Style_Crew Cab Pickup,Vehicle Style_Extended Cab Pickup,Vehicle Style_Passenger Minivan,Vehicle Style_Passenger Van,Vehicle Style_Regular Cab Pickup,Vehicle Style_Sedan,Vehicle Style_Wagon
0,2011,335.0,6.0,2.0,26,19,3916,46135,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2011,300.0,6.0,2.0,28,19,3916,40650,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2011,300.0,6.0,2.0,28,20,3916,36350,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2011,230.0,6.0,2.0,28,18,3916,29450,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,2011,230.0,6.0,2.0,28,18,3916,34500,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,2012,300.0,6.0,4.0,23,16,204,46120,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
11910,2012,300.0,6.0,

In [16]:
df.isna().sum()[df.isna().sum() > 0]

Engine HP           69
Engine Cylinders    30
Number of Doors      6
dtype: int64

In [17]:
# Fill remaining missing values
df['Engine HP'] = df['Engine HP'].fillna(df['Engine HP'].mean())

for column in ['Engine Cylinders', 'Number of Doors']:
    df[column] = df[column].fillna(df[column].mode()[0])

In [18]:
df.isna().sum().sum()

np.int64(0)

In [19]:
# Split df into X and y
y = df['MSRP'].copy()
X = df.drop('MSRP', axis=1).copy()

In [20]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [21]:
X_train.shape, X_test.shape

((8339, 104), (3575, 104))

In [22]:
# Scale X
scaler = StandardScaler()

scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

In [23]:
X_train

,Year,Engine HP,Engine Cylinders,Number of Doors,highway MPG,city mpg,Popularity,Market Category_Crossover,Market Category_Diesel,Market Category_Exotic,Market Category_Factory Tuner,Market Category_Flex Fuel,Market Category_Hatchback,Market Category_High-Performance,Market Category_Hybrid,Market Category_Luxury,Market Category_Missing,Market Category_Performance,Make_Acura,Make_Alfa Romeo,Make_Aston Martin,Make_Audi,Make_BMW,Make_Bentley,Make_Bugatti,Make_Buick,Make_Cadillac,Make_Chevrolet,Make_Chrysler,Make_Dodge,Make_FIAT,Make_Ferrari,Make_Ford,Make_GMC,Make_Genesis,Make_HUMMER,Make_Honda,Make_Hyundai,Make_Infiniti,Make_Kia,Make_Lamborghini,Make_Land Rover,Make_Lexus,Make_Lincoln,Make_Lotus,Make_Maserati,Make_Maybach,Make_Mazda,Make_McLaren,Make_Mercedes-Benz,Make_Mitsubishi,Make_Nissan,Make_Oldsmobile,Make_Plymouth,Make_Pontiac,Make_Porsche,Make_Rolls-Royce,Make_Saab,Make_Scion,Make_Spyker,Make_Subaru,Make_Suzuki,Make_Tesla,Make_Toyota,Make_Volkswagen,Make_Volvo,Engine Fuel Type_diesel,Engine Fuel Type_electric,Engine Fuel Type_flex-fuel (premium unleaded recommended/E85),Engine Fuel Type_flex-fuel (premium unleaded required/E85),Engine Fuel Type_flex-fuel (unleaded/E85),Engine Fuel Type_flex-fuel (unleaded/natural gas),Engine Fuel Type_natural gas,Engine Fuel Type_premium unleaded (recommended),Engine Fuel Type_premium unleaded (required),Engine Fuel Type_regular unleaded,Transmission Type_AUTOMATED_MANUAL,Transmission Type_AUTOMATIC,Transmission Type_DIRECT_DRIVE,Transmission Type_MANUAL,Transmission Type_UNKNOWN,Driven_Wheels_all wheel drive,Driven_Wheels_four wheel drive,Driven_Wheels_front wheel drive,Driven_Wheels_rear wheel drive,Vehicle Size_Compact,Vehicle Size_Large,Vehicle Size_Midsize,Vehicle Style_2dr Hatchback,Vehicle Style_2dr SUV,Vehicle Style_4dr Hatchback,Vehicle Style_4dr SUV,Vehicle Style_Cargo Minivan,Vehicle Style_Cargo Van,Vehicle Style_Convertible,Vehicle Style_Convertible SUV,Vehicle Style_Coupe,Vehicle Style_Crew Cab Pickup,Vehicle Style_Extended Cab Pickup,Vehicle Style_Passenger Minivan,Vehicle Style_Passenger Van,Vehicle Style_Regular Cab Pickup,Vehicle Style_Sedan,Vehicle Style_Wagon
10660,0.747903,-0.737725,-0.909115,0.643471,0.259193,0.141748,2.858336,-0.447632,-0.128761,-0.213334,-0.23317,-0.333578,-0.338880,-0.363024,-0.173248,-0.616396,1.460345,-0.463562,-0.145552,-0.018971,-0.092007,-0.171403,-0.166523,-0.079214,-0.015489,-0.130197,-0.186052,-0.325077,-0.130197,-0.232025,-0.064922,-0.078444,3.559563,-0.208371,-0.015489,-0.037962,-0.200087,-0.16112,-0.171403,-0.14031,-0.065847,-0.109611,-0.129241,-0.116149,-0.051431,-0.071995,-0.03465,-0.186741,-0.018971,-0.173981,-0.138971,-0.220307,-0.115617,-0.088634,-0.127795,-0.10562,-0.051431,-0.095904,-0.071995,-0.015489,-0.147686,-0.173615,-0.041008,-0.259994,-0.269589,-0.158357,-0.11017,-0.076881,-0.049032,-0.065847,-0.287342,-0.024494,-0.010951,-0.378508,-0.451679,0.810878,-0.238272,0.667634,-0.076881,-0.572596,-0.04245,-0.494785,-0.364714,1.220654,-0.630306,1.220654,-0.549131,-0.760958,-0.209621,-0.106199,-0.254697,-0.507895,12.747933,-0.087249,-0.269077,-0.051431,-0.336677,-0.245219,-0.236581,-0.189135,-0.103865,-0.191839,-0.584043,-0.230298
4140,0.090904,0.763695,1.324400,0.643471,-0.396376,0.030925,0.051096,-0.447632,-0.128761,-0.213334,-0.23317,-0.333578,-0.338880,-0.363024,5.772077,1.622334,-0.684770,-0.463562,-0.145552,-0.018971,-0.092007,-0.171403,-0.166523,-0.079214,-0.015489,-0.130197,5.374838,-0.325077,-0.130197,-0.232025,-0.064922,-0.078444,-0.280933,-0.208371,-0.015489,-0.037962,-0.200087,-0.16112,-0.171403,-0.14031,-0.065847,-0.109611,-0.129241,-0.116149,-0.051431,-0.071995,-0.03465,-0.186741,-0.018971,-0.173981,-0.138971,-0.220307,-0.115617,-0.088634,-0.127795,-0.10562,-0.051431,-0.095904,-0.071995,-0.015489,-0.147686,-0.173615,-0.041008,-0.259994,-0.269589,-0.158357,-0.11017,-0.076881,-0.049032,-0.065847,-0.287342,-0.024494,-0.010951,-0.378508,-0.451679,0.810878,-0.238272,0.667634,-0.076881,-0.572596,-0.04245,-0.494785,-0.364714,

In [24]:
X_train.mean()

Year                                2.161706e-15
Engine HP                          -1.261066e-16
Engine Cylinders                   -1.508167e-16
Number of Doors                    -1.328700e-17
highway MPG                        -1.890534e-18
                                        ...     
Vehicle Style_Passenger Minivan     4.558585e-17
Vehicle Style_Passenger Van        -1.704144e-18
Vehicle Style_Regular Cab Pickup   -4.758289e-17
Vehicle Style_Sedan                 7.902967e-17
Vehicle Style_Wagon                -3.195270e-18
Length: 104, dtype: float64

In [25]:
X_train.var()

Year                                1.00012
Engine HP                           1.00012
Engine Cylinders                    1.00012
Number of Doors                     1.00012
highway MPG                         1.00012
                                     ...   
Vehicle Style_Passenger Minivan     1.00012
Vehicle Style_Passenger Van         1.00012
Vehicle Style_Regular Cab Pickup    1.00012
Vehicle Style_Sedan                 1.00012
Vehicle Style_Wagon                 1.00012
Length: 104, dtype: float64

#### Dimensionality Reduction

We have a very high number of columns for our purposes, so let's reduce the data using the PCA to have only 100 columns (to increase training speed).

In [26]:
n_components = 100

pca = PCA(n_components=n_components)
pca.fit(X_train)

X_train = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])
X_test = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=["PC" + str(i) for i in range(1, n_components + 1)])

In [27]:
X_train

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100
10660,-2.207051,-1.731002,-0.672992,0.735962,0.089681,0.805733,3.850911,2.393780,-2.574375,-0.234215,1.389818,1.121890,0.735338,0.405410,1.692671,-0.796962,-2.080482,-0.487751,-0.136007,1.663321,-1.066629,3.044523,0.692072,-1.352602,-0.559076,5.245817,-1.781763,1.564536,-0.082835,0.355929,-0.403278,-0.273223,2.550713,-0.147681,1.219848,1.237814,0.205473,-2.558826,-2.008032,-1.331898,0.913178,1.140081,0.953897,0.833307,0.070478,1.143713,2.782761,1.266500,-0.504445,1.194861,-0.203016,3.098487,-2.251272,4.329809,0.419076,-0.223972,0.511519,3.947264,-0.023448,-2.031068,-0.316026,-0.135716,-0.127001,-1.244754,0.923157,-0.943006,0.485847,0.961867,0.035192,0.031170,-0.346470,0.772689,-0.524787,0.062135,-0.895099,0.240055,-0.547814,-0.066982,-0.198027,-0.188397,0.916599,0.217476,0.578359,-0.167068,-0.124812,-0.046364,0.180806,0.028738,0.186745,0.062465,0.082588,-0.106493,-0.057495,0.158484,0.012805,-0.001758,1.665255e-14,1.327654e-15,-1.903289e-13,-2.693363e-13
4140,1.693145,-1.095496,1.816292,0.587013,0.390592,-0.147049,-0.863307,-0.279210,-0.840215,-2.233810,-1.623650,0.001733,1.589230,-0.364651,1.116328,-0.150461,2.715235,0.723604,1.458263,1.171158,-1.131596,1.096160,-0.771912,0.749367,1.359572,0.530613,1.404011,1.286141,0.496152,-1.979677,-0.784886,1.222193,-0.251269,-1.243484,0.634861,1.205381,-1.027579,1.379532,0.016777,0.441020,1.182603,-0.649188,-0.227331,0.269929,-0.383046,0.173592,0.689515,-0.345789,0.009572,0.019422,0.013786,-0.717416,-0.648711,0.035128,0.975285,0.466501,-0.003924,-1.178650,1.479297,1.180854,-0.255332,1.707660,-0.715125,-2.089821,0.631304,0.526881,0.027521,0.086603,-0.375033,2.110204,-0.578617,0.211776,0.155274,0.646671,-1.628802,-0.010902,0.058245,-0.553166,0.951555,-0.283118,0.932163,-0.343689,-0.446648,-0.539809,0.631569,-0.560820,0.318433,0.001338,-0.015725,-0.162709,0.012185,-0.171557,-0.527907,-0.086115,-0.119489,0.000046,-1.939498e-13,-2.233815e-13,1.256431e-14,2.600180e-13
11414,-0.270184,-2.718327,-0.045718,1.041036,-0.157851,2.415403,-1.283411,2.186850,0.487343,1.898638,0.472812,0.326319,-1.745511,-1.040533,-0.172832,3.067100,-2.228750,-0.311949,1.674115,-1.237651,1.421185,0.780201,1.137327,2.139966,1.389465,-0.056930,1.531619,-2.336277,-0.095538,0.190968,-0.219467,1.312092,-0.833541,-0.106568,-0.880883,-1.804052,-0.756324,-1.070964,-2.825582,1.867682,1.210335,-0.563787,-0.151830,-1.553503,-0.338508,0.160139,0.124381,0.436240,-0.133952,0.210392,0.026390,-0.682861,-1.306553,0.594717,-0.264051,0.513695,-1.777995,-0.289616,-0.886084,-0.277679,-1.790867,1.350405,-0.411589,-0.281302,-0.517420,0.354359,0.744730,0.393120,-0.877787,0.217220,-0.029268,-0.043208,0.823412,0.481189,-1.309710,0.584854,-0.708625,-0.532766,-0.347208,-0.082384,0.038115,0.126218,-1.219319,-0.328798,0.597440,1.042681,-1.148890,0.056568,-0.884762,-1.705906,-0.068591,0.141449,0.006712,0.327975,-0.003728,0.004031,3.059489e-13,9.973505e-14,-2.936630e-13,-1.112960e-13
5119,2.743703,2.306195,0.328075,-0.906494,0.925906,1.865861,-0.827424,-1.592953,-0.492798,-0.467818,-0.528387,-1.048576,-0.436110,1.692417,-0.670348,-1.807972,-0.277314,-0.374270,-0.566143,-0.440075,0.839806,-0.631305,-0.444690,-0.014224,0.228230,0.245512,0.023492,-1.157936,-1.312686,-1.147275,0.534970,-0.002933,-1.206467,0.516525,0.682086,-1.401310,1.563724,1.165014,-1.600857,-0.873710,-1.243872,0.755237,0.685802,0.020445,-0.565897,0.454184,0.143776,-0.192418,0.217947,-0.282604,-0.232143,0.029611,-0.922444,1.436279,0.001324,-0.283562,0.184244,-0.452941,-0.201598,0.684978,0.972516,-0.10543

### Training

In [28]:
models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor()
}

In [29]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.


C:\Users\dukea_6wbfoy1\.conda\envs\tfenv\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [30]:
X_train.var()

PC1      6.272760e+00
PC2      4.775776e+00
PC3      4.388434e+00
PC4      3.298823e+00
PC5      2.620420e+00
             ...     
PC96     6.054559e-04
PC97     8.836150e-26
PC98     9.433111e-26
PC99     3.940199e-25
PC100    4.698604e-26
Length: 100, dtype: float64

### Results

In [32]:
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))

                     Linear Regression R^2 Score: 0.79505
 Linear Regression (L2 Regularization) R^2 Score: 0.79506
 Linear Regression (L1 Regularization) R^2 Score: 0.79508


C:\Users\dukea_6wbfoy1\.conda\envs\tfenv\lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


                   K-Nearest Neighbors R^2 Score: 0.76867
                        Neural Network R^2 Score: 0.48400
Support Vector Machine (Linear Kernel) R^2 Score: -0.25656
   Support Vector Machine (RBF Kernel) R^2 Score: -0.02933
                         Decision Tree R^2 Score: 0.87070
                         Random Forest R^2 Score: 0.87457
                     Gradient Boosting R^2 Score: 0.86735
